In [1]:
#paths required
datafolder_p = r'D:\18-DS\github\SDSHL\data\processed'
datafolder_e = r'D:\18-DS\github\SDSHL\data\external'
datafolder_i = r'D:\18-DS\github\SDSHL\data\internal'
modelfolder  = r'D:\18-DS\data\models'

modelfolder_w2v = modelfolder + r'\word2vec'

filepath_train  = datafolder_p+ r'\2-train.csv'
filepath_fulldata = datafolder_p+ r'\2-Hinglish_Sarcasm_Clean.csv'

In [2]:
from gensim.models import Word2Vec
import pandas as pd
import nltk
import numpy as np

In [3]:
# Set values for various parameters
feature_size = 15    # Word vector dimensionality  
window_context = 20  # Context window size                                                                                    
min_word_count = 1   # Minimum word count                        
sample = 1e-3        # Downsample setting for frequent words
sg = 1               # skip-gram model

In [4]:
df= pd.read_csv(filepath_train, sep='\t')
corpus = list(df['sentence'])
tokenized_corpus = [nltk.word_tokenize(doc) for doc in corpus]
df['sentence_tkn'] = tokenized_corpus


#This is required to create word embedding of full dataset
df_full =pd.read_csv(filepath_fulldata, usecols=['ID','label','sentence'], sep="\t", index_col="ID")
corpus_full = list(df_full['sentence'])
tokenized_corpus_full = [nltk.word_tokenize(doc) for doc in corpus_full]
df_full['sentence_tkn'] = tokenized_corpus_full

In [5]:
df.shape

(1800, 5)

In [6]:
#THIS STEP TAKS TOO MUCH TIME.

embedding_model = Word2Vec(tokenized_corpus, size=feature_size, 
                                  window=window_context, min_count = min_word_count,
                                  sg=sg, sample=sample, iter=5000)

In [7]:
embedding_model.save(modelfolder_w2v + r'\model_w2v.bin')

In [8]:
def average_word_vectors(words, model, vocabulary, num_features):
    feature_vector = np.zeros((num_features,),dtype="float64")
    nwords = 0.
    for word in words:
        if word in vocabulary: 
            nwords = nwords + 1.
            feature_vector = np.add(feature_vector, model.wv[word])
    if nwords:
        feature_vector = np.divide(feature_vector, nwords)
    return feature_vector

def averaged_word_vectorizer(corpus, model, num_features):
    vocabulary = set(model.wv.index2word)
    features = [average_word_vectors(tokenized_sentence, model, vocabulary, num_features)
                    for tokenized_sentence in corpus]
    return np.array(features)

In [9]:
# get document level embeddings
w2v_doc_features = averaged_word_vectorizer(corpus=tokenized_corpus_full, model=embedding_model,
                                             num_features=feature_size)
df_w2v = pd.DataFrame(w2v_doc_features)

In [18]:
df_w2v['label']=df_full['label']
df_w2v = df_w2v.set_index( df_full.index)

In [26]:
df_w2v.to_csv(datafolder_p+r'\embedding_w2v.csv')

In [27]:
df_w2v

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,label
ID,,,,,,,,,,,,,,,,
1,0.610926,-0.033076,-0.560991,-0.024154,0.549569,-0.205838,-0.835064,-0.144946,0.297129,-0.394314,-0.094137,-0.215634,-0.954738,-0.810391,0.284191,1
5,0.528000,-0.354342,-0.392159,0.871123,0.113215,-0.329450,-0.937927,-0.194318,-0.193804,-0.373432,-0.287409,0.114795,-0.260780,0.347092,0.232011,1
12,0.043671,-0.084298,-0.007432,1.241895,-0.034132,0.022502,-0.566449,0.583137,0.276687,-0.005595,-0.764145,-0.599398,-1.002398,-0.902235,0.000543,1
13,1.063604,-0.658784,-0.246466,0.083243,1.599032,0.469778,-1.082058,-0.378334,-0.136459,-0.245181,0.148344,-0.633841,-0.914052,-1.454631,-0.173933,1
26,0.059589,-1.227684,-0.239895,0.450532,-0.584627,-0.945401,-1.297013,1.301371,-0.310257,-2.060586,-1.337901,0.038378,-0.314948,-0.546724,0.425531,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9347,-0.236982,-0.220310,-0.759061,0.423561,-0.111851,-0.018247,-0.855448,-0.039831,0.705891,-0.787966,0.034041,-0.658152,-0.046181,-0.569668,0.462961,1
9348,0.287712,-0.629669,-0.517868,1.148785,0.374133,-1.200779,-0.890466,-0.037295,1.113050,-0.247304,0.300192,-1.095296,-1.383915,0.008150,0.313038,1
9349,0.435490,-0.930899,-0.452073,0.827819,0.460287,-0.376886,-0.837968,-0.351987,-0.779221,-1.034075,-0.438819,-1.994503,-1.401430,-0.119565,0.097246,1


# Experiment Area

In [ ]:
%lsmagic

In [ ]:
%alias

In [ ]:
%time

In [ ]:
%pwd

In [ ]:
%set_env

In [ ]:
%pip

In [ ]:
!pip list

In [ ]:
!pip wheel pandas

In [ ]:
%env

In [ ]:
%timeit

In [ ]:
 %who

In [ ]:
type(embedding_model)